# 7.f Sieci LSTM

Pracujemy na zbiorze danych z konkursu KLEJ - Allegro Reviews. Konstruujemy klasyfikator recenzji pod względem wydźwięku, oparty o reprezentacje BiLSTM (dwukierunkowy LSTM).

Ponieważ LSTM uczy się wolno, skorzystamy z batchingu - podziału całego zbioru danych na wsady danych po kilka przykładów naraz. Batching pozwala na paralelizację obliczeń, przyśpieszając uczenie.

Jako że pracujemy z danymi sekwencyjnymi, a tensory muszą mieć stały rozmiar w każdym wymiarze, zastosujemy padding, czyli uzupełnianie krótszych dokumentów wektorami zerowymi, do określonej długości.



#### Import pakietów

In [ ]:
import os
import random
import numpy
import torch
from tqdm.notebook import tqdm

#### Ładowanie danych

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/lstm_data.zip?raw=true
!mv lstm_data.zip?raw=true lstm_data.zip
!unzip lstm_data.zip

In [ ]:
import re


tokenizer = re.compile(r"[\w]+")

simplification = {"1.0":"1.0", # mapowanie 5-stopniowej skali ocen, na 3-stopniową
                 "2.0":"1.0",
                 "3.0":"2.0",
                 "4.0":"3.0",
                 "5.0":"3.0",}

def load_data(path, simplify=False): # simplify określa czy korzystamy z 5, czy z 3-stopniowej skali
    with open(path) as f:
        data = f.read()
    doc_pairs = data.split("\n")[:-1]
    out = []
    for doc_pair in doc_pairs[1:]:
        text, label = doc_pair.split("\t")
        if simplify:
            label = simplification[label]
        if len(tokenizer.findall(text)) > 0: # odfiltrowujemy anomalie
            out.append((text, int(float(label) -1)))
    if simplify:
        num_labels = 3
    else:
        num_labels = 5
    return out, num_labels

train_data, num_labels = load_data(os.path.join("lstm_data", "train.tsv"), True)
dev_data, _ = load_data(os.path.join("lstm_data", "dev.tsv"), True)
label_list = list(range(0, num_labels))

print("Kategorie dla klasyfikatora: ", label_list, "\n")
print("Długość zbioru treningowego (liczona w dokumentach): ", len(train_data))
print("Przykładowy dokument:")
print(train_data[100])
doc_lens = [len(tokenizer.findall(t)) for t, l in train_data]
print("Średnia długość dokumentu: ", sum(doc_lens)/len(doc_lens))

#### Ładowanie modelu fasttext

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
!gunzip cc.pl.300.bin.gz
!python3 -m pip install fasttext

In [ ]:
import fasttext
VEX = fasttext.load_model("cc.pl.300.bin")
num_feats = VEX.get_dimension()

def w2v(form):
    try:
        return VEX.get_word_vector(form)
    except KeyError:
        return numpy.zeros((num_feats,))

#### Konstrukcja sieci

In [ ]:
from torch import nn, cat, tanh
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(0.5) # regularyzacja
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        # parametry określają: wymiar wejścia, liczbę jednostek
        # wykorzystanie dwu warstw dla każdego z kierunków
        # oraz miejsce współrzędnej reprezentującej numer przykładu w batchu
        
        self.dense = nn.Linear(hidden_size*2, output_size) # klasyfikator, 
        # 2*hidden_size, ponieważ mamy sieć dwukierunkową
        self.softmax = nn.LogSoftmax(dim=1)

    def init_state(self, batch_size): # batch_size to liczba przykładów w batchu
        # podajemy stany ukryte dla całego batcha
        state = torch.zeros(2, batch_size, self.hidden_size) # 2, ponieważ mamy sieć dwukierunkową
        cell = torch.zeros(2, batch_size, self.hidden_size)
        return state, cell

    def forward(self, input, lengths):
        batch_size = input.shape[0]
        zero_hidden, zero_cell = self.init_state(batch_size)
        
        # przed podaniem danych do warstwy rekurencyjnej wykorzystujemy
        # funkcjonalności pytorch ułatwiające pracę z padowanymi batchami danych
        packed_input = pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=True)
        # w tym celu musimy podać 
        #    1. tensor danych
        #    2. listę długości każdej z sekwencji (recenzji) przed paddingiem
        #    3. określić miejsce współrzędnej reprezentującej numer przykładu w batchu
        #    4. czy dane są już posortowane pod względem długości
        packed_output, _ = self.lstm(packed_input, (zero_hidden, zero_cell)) 
        # podajemy spakowany input oraz zerowe stany ukryte dla całego batcha
        output, lengths = pad_packed_sequence(packed_output, batch_first=True)
        # odpakowujemy output z warstwy LSTM
        
        aggregated = self.dropout(output.sum(1).div(lengths.unsqueeze(1)))
        # zamiast pobierać ostatni stan ukryty (pamięć po całej sekwencji danych)
        # do reprezentacji całej sekwencji wykorzystujemy wektor uśredniony
        # (ignorując padding)
        # przed podaniem do klasyfikatora aplikujemy dropout aby uniknąć przeuczenia
        output = self.softmax(self.dense(aggregated)) # klasyfikacja
        return output

### Funkcje do pracy z danymi

Definiujemy funkcje do przygotowywania batchy danych, trenowania oraz testowania na batchu, i przetwarzania zwykłych stringów.

In [ ]:
from torch import tensor


padding_vector = numpy.zeros((num_feats,)) 
# wektor zer do paddingu (uzupełniania danych do jednej długości)


def datapoints_to_batch(datapoints, maxlen):
    # zamienia listę przykładów w batch do podania do sieci
    batchsize = len(datapoints)
    vex = []
    lengths = [] # tutaj zapisujemy długość sekwencji po obcięciu, przed paddingiem
    labels = []
    for datapoint in datapoints:
        string, label = datapoint
        vec = [w2v(form) for form in tokenizer.findall(string)] # wektoryzacja
        tok_num = len(vec) # liczba tokenów w przykładzie
        if tok_num > maxlen: 
            vec = vec[:maxlen] # obcinanie zbyt długich sekwencji(truncation)
            tok_num = maxlen
        lengths.append(tok_num)
        vex.append(vec)
        labels.append(label)

    # sortowanie danych pod względem długości
    indices = [(i,l) for i,l in enumerate(lengths)]
    indices = [i for (i, l) in sorted(indices, key=lambda x:x[1], reverse=True)]
    vex = [vex[i] for i in indices]
    lengths = [lengths[i] for i in indices]
    labels = [labels[i] for i in indices]

    maxlen = max(lengths) # jeśli w batchu nie ma dłuższych niż maxlen przykładów, ograniczamy długość paddingu
    for vec in vex: # padding początku sekwencji (prepadding)
        while len(vec) < maxlen:
            vec.insert(0, padding_vector) # uzupełniamy sekwencje wektorem zer
    X = torch.tensor(vex, dtype=torch.float32)
    Y = torch.tensor([l for l in labels])
    return X, Y, lengths


def train_on_batch(model, criterion, optimizer, X, Y, lengths):
    # nie aktualizujemy wag własnoręcznie, tylko korzystamy z optimizera
    model.train()
    optimizer.zero_grad()# zerujemy gradienty w optimizerze (a nie w modelu)
    output = model(X, lengths)
    loss = criterion(output, Y)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # obcinanie gradientów (dla uniknięcia eksplozji gradientów)
    optimizer.step() # aktualizacja wag
    return loss.item()


def test_on_batch(model, X, Y, lengths):
    model.eval()
    output = model(X, lengths)
    decision = output.topk(1).indices.squeeze() # pobieramy decyzje klasyfikatora
    equal = decision == Y # tensor określający które decyzje zostały poprawnie wydane
    correct = sum(equal).item() # sumowanie liczby poprawnych decyzji
    return correct, decision
    
def predict_sentence(model, sentence, maxlen):
    batchsize = 1
    vex = []
    lengths = []
    labels = []

    vec = vec = [w2v(form) for form in tokenizer.findall(sentence)] # wektoryzacja
    tok_num = len(vec)
    if tok_num > maxlen: # obcinanie sekwencji
        vec = vec[:maxlen]
        tok_num = maxlen
    lengths.append(tok_num) 
    vex.append(vec)
    X = torch.tensor(vex, dtype=torch.float32)
    
    model.eval()
    output = model(X, lengths) # przepuszczenie przez model
    decision = output.topk(1).indices.squeeze() # pobranie decyzji modelu
    return decision

### Przykładowe przejście przez sieć

In [ ]:
batch = train_data[0:16]
# 16 przykładów (recenzji) jako batch (wsad) danych
# batching paralelizuje obliczanie kilku przykładów, przyspieszając uczenie

X, Y, lengths = datapoints_to_batch(batch, 30)
# przetworzenie wsadu, obcinamy/padujemy sekwencje do 100 tokenów

print("Kształt wejścia do sieci: ", X.shape)
print("Lista długości sekwencji: ", lengths)
# liczba sekwencji w batchu, liczba tokenów w sekwencji, liczba cech dla tokenu
print("Reprezentacja przykładowej sekwencji (z paddingiem):\n", X[-1])
print("Kształt wzorcowego wyjścia z sieci:", Y.shape)
# 16 liczb całkowitych (etykiet) dla każdej z sekwencji

model = LSTMModel(num_feats, 40, num_labels) # przygotowanie modelu z 40 jednostkami LSTM

batch_size = X.shape[0]

zero_hidden, zero_cell = model.init_state(batch_size)
print("Kształt stanu ukrytego dla warstwy lstm w kroku 0:",zero_hidden.shape)
# liczba kierunków, liczba sekwencji w batchu, liczba cech (jednostek lstm)

# pakowanie danych
packed_input = pack_padded_sequence(X, lengths, batch_first=True, enforce_sorted=True)
print("\nObiekt PackedSequence:\n", packed_input, "\n\n")
packed_output, _ = model.lstm(packed_input,(zero_hidden, zero_cell))
output, lengths = pad_packed_sequence(packed_output, batch_first=True)
print("Kształt rozpakowanego wyjścia z sieci:", output.shape)
# liczba sekwencji w batchu, liczba tokenów, liczba cech
mean = output.sum(1).div(lengths.unsqueeze(1)) # sumowanie reprezentacji tokenów, i dzielenie przez długość sekwencji
dropped = model.dropout(mean)
print("Kształt zagregowanej reprezentacji sekwencji", dropped.shape)
print("Przykładowe wejście do klasyfikatora, po aplikacji dropoutu", dropped[0])
# liczba sekwencji w batchu, liczba cech

output = model.softmax(model.dense(dropped))
print("Kształt wyjścia z sieci:", output.shape)
# liczba sekwencji w batchu, liczba możliwych etykiet

### Trening modelu

In [ ]:
model = LSTMModel(num_feats, 20, num_labels) # 20 jednostek, prosty model ma mniejszą tendencję do przeuczenia
criterion = torch.nn.NLLLoss()
learning_rate = 0.005
epochs = 8
batch_size = 10 # liczba przykładów we wsadzie danych, im wyższa tym szybsze uczenie, ale gorsze wyniki
maxlen = 50 # maksymalna długość przykładu
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Do aktualizacji wag wykorzystamy algorytm Adam


# obliczanie liczby batchy
num_train_batches = len(train_data) // batch_size + int(bool(len(train_data) % batch_size)) 
num_dev_batches = len(dev_data) // batch_size + int(bool(len(dev_data) % batch_size))



best_acc = 0 # zapisujemy wynik najlepszego modelu
for epoch in range(epochs):
    print("epoch no ", epoch + 1)
    random.shuffle(train_data) # tasowanie danych
    total_loss = 0
    for n in tqdm(range(num_train_batches)): # iterujemy po numerach batchy
        datapoints = train_data[n*batch_size:(n+1)*batch_size] 
        tb = datapoints_to_batch(datapoints, maxlen)# przygotowanie wsadu danych
        X, Y, lengths = tb
        loss = train_on_batch(model, criterion, optimizer, X, Y, lengths) # trening
        total_loss += loss
    print(total_loss)

    # ewaluacja na devsecie
    with torch.no_grad():
        total = 0
        correct = 0
        for n in range(num_dev_batches):
            datapoints = dev_data[n*batch_size:(n+1)*batch_size]
            tb = datapoints_to_batch(datapoints, maxlen)
            X, Y, lengths = tb
            result, _ = test_on_batch(model, X, Y, lengths)
            total += batch_size
            correct += result
        acc = (correct/total) * 100
        accuracy = "{:4.2f}%".format((acc))
        print("dev accuracy: ", accuracy)
        if acc > best_acc:
            # zapisywanie najlepszego modelu
            best_acc = acc
            torch.save(model, "best_model.model")
            


#### Augmentacja danych
Ponieważ istotnie skracamy dane, warto rozważyć rozbicie każdego przykładu na kilka krótszych.

In [ ]:
def augment_data(data, maxlen):
    augmented = []
    for example in data:
        string, label = example
        tokens = tokenizer.findall(string)
        num_new_examples = len(tokens) // maxlen + int(bool(len(tokens) % maxlen))
        for n in range(num_new_examples):
            new_tokens = tokens[n*maxlen:(n+1)*maxlen]
            new_string = " ".join(new_tokens)
            augmented.append((new_string, label))
    return augmented

len_before = len(train_data)
train_data = augment_data(train_data, 50)
len_after = len(train_data)
print("{} -> {}".format(len_before, len_after)) 

#### Zastosowanie na dokumencie

In [ ]:
doc = "Uchwyt dobrze trzyma telefon. Ramiona mają spory zakres regulacji. \
       Element, w którym osadza się telefon jest obrotowy, dzięki czemu telefon\
       można ustawić pod dowolnym kątem. Fajnie przemyślane mocowanie do kratki\
       - nie ma możliwości by się sam odczepił. Dodatkowo uchwyt posiada na dole\
       dwa wsporniki, które moim zdaniem zmniejszają obciążenie samej kratki nawiewu\
       (zmniejszają ryzyko wyłamania kratki). Mocowanie do kratki nie jest sztywne,\
       w związku z czym wydaje mi się, że może lekko podskakiwać na wyboistych\
       drogach. Uchwyt jest wykonany w większości z plastku, ale mimo wszystko\
       wydaje się solidny. Używam go od dwóch miesięcy i spełnia swoją funkcję\
       (telefon jeszcze mi nie wypadł). Moim zdaniem jest ok. Jestem zadowolony."

result = predict_sentence(model, doc, 50)
print(result)

#### Macierz konfuzji
Macierz konfuzji pozwala zobaczyć sposób w jakimodel traktuje każdą z kategorii, i zdiagnozować problemy z niektórymi klasami.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


gold = []
sys = []
for n in tqdm(range(num_dev_batches)):
            datapoints = dev_data[n*batch_size:(n+1)*batch_size]
            tb = datapoints_to_batch(datapoints, maxlen)
            X, Y, lengths = tb
            result, predictions  = test_on_batch(model, X, Y, lengths)
            gold.append(Y)
            sys.append(predictions)

sys = torch.cat(sys).tolist()
gold = torch.cat(gold).tolist()

cm = confusion_matrix(gold, sys, normalize="true")

plt.imshow(cm, cmap='hot')
plt.xlabel("Prediction")
plt.ylabel("True label")
plt.yticks([0,1,2])
plt.xticks([0,1,2])
plt.show()